In [1]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import talib
import mpl_finance as mpf
%matplotlib inline
import seaborn as sns
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
import csv
import time
plt.rcParams['font.family']='SimHei' # 顯示中文('SimHei' for MacOS)

'''today = date.today()
print(today.strftime("%Y%m%d"))'''

fina=input('輸入個股代碼(ex:1101)：')
date=input('輸入日期(ex:2018/10/01)：')

inday=datetime.datetime.strptime(date,"%Y/%m/%d")
inday=datetime.datetime.strptime(inday.strftime('%Y/%m/'+'01'),"%Y/%m/%d")
ago=[(inday+relativedelta(years=-1)+relativedelta(months=+i)).strftime('%Y%m%d') for i in range(13) ]
ago

month = ago[0]
data = {
    'response':'json',
    'date': month,
    'stockNo':fina,
}
dres = requests.get('https://www.twse.com.tw/exchangeReport/STOCK_DAY?', params=data)
result = json.loads(dres.text)
df1 = pd.DataFrame(result['data'])
df1.columns = result['fields']
print(month)
time.sleep(3)
for month in ago[1:13] :
    data = {
        'response':'json',
        'date': month,
        'stockNo':fina,
    }
    dres = requests.get('https://www.twse.com.tw/exchangeReport/STOCK_DAY?', params=data)
    result = json.loads(dres.text)
    tmp = pd.DataFrame(result['data'])
    tmp.columns = result['fields']
    df1=df1.append(tmp).reset_index(drop=True)
    print(month)
    time.sleep(3)

df1['開盤價']=df1['開盤價'].str.replace(',','').astype(dtype=float,errors='ignore')
df1['最高價']=df1['最高價'].str.replace(',','').astype(dtype=float,errors='ignore')
df1['最低價']=df1['最低價'].str.replace(',','').astype(dtype=float,errors='ignore')
df1['收盤價']=df1['收盤價'].str.replace(',','').astype(dtype=float,errors='ignore')
df1['成交股數']=df1['成交股數'].str.replace(',','').astype(dtype=float,errors='ignore')
df1['成交金額']=df1['成交金額'].str.replace(',','').astype(dtype=float,errors='ignore')
df1['成交筆數']=df1['成交筆數'].str.replace(',','').astype(dtype=float,errors='ignore')
df1['漲跌價差']=df1['漲跌價差'].str.replace('+','')
df1['漲跌價差']=df1['漲跌價差'].str.replace('X','').astype(dtype=float,errors='ignore')
pd.set_option('display.max_rows', 1000)
indexNames = df1[ df1['成交金額'] == 0 ].index
df1 = df1.drop(indexNames)
df1=df1.reset_index(drop=True)
df1

ModuleNotFoundError: No module named 'requests'

In [ ]:
def RSI(n,index):
    mean_up=0
    mean_down=0
    data=df1.loc[index-(n-1):index+1,'漲跌價差']
    for i in data :
        if float(i)>0:
            mean_up+=float(i)
        else:
            mean_down-=float(i)
    mean_up/=n
    mean_down/=n
    if mean_down == 0 :
        return np.nan
    else :
        rs=mean_up/mean_down
        rsi=100*rs/(1+rs)
        return rsi
def KD(n,index,K_before,D_before):
    Mtoday=float(df1.at[index,'收盤價'])
    data=df1.loc[index-(n-1):index+1,'收盤價']
    Max=float(max(data))
    Min=float(min(data))
    RSV=(Mtoday-Min)/(Max-Min) *100
    K=K_before*0.6667+RSV*0.3333
    D=D_before*0.6667+K*0.3333
    return (K,D)
#RSI_6
N_day=6
rsi=RSI(N_day,N_day-1)
df1.loc[N_day-1,'RSI_6']=rsi
for i in range(N_day,len(df1)):
    rsi=RSI(N_day,i)
    df1.loc[i,'RSI_6']=rsi
#RSI_12
N_day=12
rsi=RSI(N_day,N_day-1)
df1.loc[N_day-1,'RSI_12']=rsi
for i in range(N_day,len(df1)):
    rsi=RSI(N_day,i)
    df1.loc[i,'RSI_12']=rsi

for i in range(len(df1)-1):
    dif=float(df1.loc[i+1,'最高價'])-float(df1.loc[i,'最高價'])
    if dif > 0:
        df1.loc[i,'Ans']=1
    else:
        df1.loc[i,'Ans']=-1
df1

In [4]:
#KD_5
N_day=5
K,D=KD(N_day,N_day-1,50,50)
df1.loc[N_day-1,'K']=K
df1.loc[N_day-1,'D']=D
for i in range(N_day,len(df1)):
    K,D=KD(N_day,i,K,D)
    df1.loc[i,'K']=K
    df1.loc[i,'D']=D
for i in range(12,len(df1)):
    Yesterday=df1.loc[i-1,'RSI_12']-df1.loc[i-1,'RSI_6']
    Today    =df1.loc[i,'RSI_12']-df1.loc[i,'RSI_6']
    Product = Yesterday*Today
    if Product < 0 or (abs(Product-0)<0.0001) :
        if Yesterday > 0:
            df1.loc[i,'RSI_DIF']=1
        else :
            df1.loc[i,'RSI_DIF']=-1
    else :
        df1.loc[i,'RSI_DIF']=0
df1

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,RSI_6,RSI_12,K,D,RSI_DIF
0,106/04/05,7385595.0,2.727916e+08,36.40,37.20,36.40,37.05,0.75,3091.0,NaN,NaN,NaN,NaN,NaN
1,106/04/06,5074098.0,1.890247e+08,37.20,37.45,37.10,37.30,0.25,2430.0,NaN,NaN,NaN,NaN,NaN
2,106/04/07,3786727.0,1.395635e+08,36.95,37.25,36.50,37.15,-0.15,1496.0,NaN,NaN,NaN,NaN,NaN
3,106/04/10,978271.0,3.619003e+07,37.15,37.15,36.85,37.00,-0.15,619.0,NaN,NaN,NaN,NaN,NaN
4,106/04/11,3063366.0,1.134365e+08,37.00,37.30,36.90,36.90,-0.10,1280.0,NaN,NaN,37.038333,45.679876,NaN
5,106/04/12,4321948.0,1.596020e+08,37.25,37.50,36.70,36.85,-0.05,1886.0,64.516129,NaN,30.753457,40.704901,NaN
6,106/04/13,2235480.0,8.236290e+07,36.90,37.05,36.75,36.75,-0.10,1101.0,23.809524,NaN,33.322560,38.244367,NaN
7,106/04/14,2230761.0,8.158226e+07,36.60,36.75,36.40,36.50,-0.25,1074.0,0.000000,NaN,22.216151,32.902162,NaN
8,106/04/17,2460829.0,8.975964e+07,36.65,36.70,36.35,36.50,0.00,1750.0,0.000000,NaN,18.514841,28.106868,NaN
9,106/04/18,3836886.0,1.397897e+08,36.60,36.65,36.25,36.45,-0.05,1509.0,0.000000,NaN,35.008245,30.407097,NaN


In [5]:
#BIAS
def BIAS(n,index):
    Mtoday=float(df1.at[index,'收盤價'])
    data=np.asarray(df1.loc[index-(n-1):index+1,'收盤價'].astype(float))
    Mean=np.mean(data)
    ans=100*(Mtoday-Mean)/Mean
    return ans
N_day=6
bias=BIAS(N_day,N_day-1)
df1.loc[N_day-1,'BIAS']=bias
for i in range(N_day,len(df1)):
    bias=BIAS(N_day,i)
    df1.loc[i,'BIAS']=bias
df1

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,RSI_6,RSI_12,K,D,RSI_DIF,BIAS
0,106/04/05,7385595.0,2.727916e+08,36.40,37.20,36.40,37.05,0.75,3091.0,NaN,NaN,NaN,NaN,NaN,NaN
1,106/04/06,5074098.0,1.890247e+08,37.20,37.45,37.10,37.30,0.25,2430.0,NaN,NaN,NaN,NaN,NaN,NaN
2,106/04/07,3786727.0,1.395635e+08,36.95,37.25,36.50,37.15,-0.15,1496.0,NaN,NaN,NaN,NaN,NaN,NaN
3,106/04/10,978271.0,3.619003e+07,37.15,37.15,36.85,37.00,-0.15,619.0,NaN,NaN,NaN,NaN,NaN,NaN
4,106/04/11,3063366.0,1.134365e+08,37.00,37.30,36.90,36.90,-0.10,1280.0,NaN,NaN,37.038333,45.679876,NaN,NaN
5,106/04/12,4321948.0,1.596020e+08,37.25,37.50,36.70,36.85,-0.05,1886.0,64.516129,NaN,30.753457,40.704901,NaN,-4.054054e-01
6,106/04/13,2235480.0,8.236290e+07,36.90,37.05,36.75,36.75,-0.10,1101.0,23.809524,NaN,33.322560,38.244367,NaN,-4.643064e-01
7,106/04/14,2230761.0,8.158226e+07,36.60,36.75,36.40,36.50,-0.25,1074.0,0.000000,NaN,22.216151,32.902162,NaN,-8.344654e-01
8,106/04/17,2460829.0,8.975964e+07,36.65,36.70,36.35,36.50,0.00,1750.0,0.000000,NaN,18.514841,28.106868,NaN,-5.643121e-01
9,106/04/18,3836886.0,1.397897e+08,36.60,36.65,36.25,36.45,-0.05,1509.0,0.000000,NaN,35.008245,30.407097,NaN,-1.565251e-01


In [7]:
#WMR
def WMR(n,index):
    Mtoday=float(df1.at[index,'收盤價'])
    H=np.asarray(df1.loc[index-(n-1):index+1,'最高價'].astype(float))
    L=np.asarray(df1.loc[index-(n-1):index+1,'最低價'].astype(float))
    Hn=np.max(H)
    Ln=np.min(L)
    ans=100*(Hn-Mtoday)/(Hn-Ln)
    return ans
N_day=6
wmr=WMR(N_day,N_day-1)
df1.loc[N_day-1,'WMR']=wmr
for i in range(N_day,len(df1)):
    wmr=WMR(N_day,i)
    df1.loc[i,'WMR']=wmr
df1

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,RSI_6,RSI_12,K,D,RSI_DIF,BIAS,WMR
0,106/04/05,7385595.0,2.727916e+08,36.40,37.20,36.40,37.05,0.75,3091.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,106/04/06,5074098.0,1.890247e+08,37.20,37.45,37.10,37.30,0.25,2430.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,106/04/07,3786727.0,1.395635e+08,36.95,37.25,36.50,37.15,-0.15,1496.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,106/04/10,978271.0,3.619003e+07,37.15,37.15,36.85,37.00,-0.15,619.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,106/04/11,3063366.0,1.134365e+08,37.00,37.30,36.90,36.90,-0.10,1280.0,NaN,NaN,37.038333,45.679876,NaN,NaN,NaN
5,106/04/12,4321948.0,1.596020e+08,37.25,37.50,36.70,36.85,-0.05,1886.0,64.516129,NaN,30.753457,40.704901,NaN,-4.054054e-01,59.090909
6,106/04/13,2235480.0,8.236290e+07,36.90,37.05,36.75,36.75,-0.10,1101.0,23.809524,NaN,33.322560,38.244367,NaN,-4.643064e-01,68.181818
7,106/04/14,2230761.0,8.158226e+07,36.60,36.75,36.40,36.50,-0.25,1074.0,0.000000,NaN,22.216151,32.902162,NaN,-8.344654e-01,86.956522
8,106/04/17,2460829.0,8.975964e+07,36.65,36.70,36.35,36.50,0.00,1750.0,0.000000,NaN,18.514841,28.106868,NaN,-5.643121e-01,80.000000
9,106/04/18,3836886.0,1.397897e+08,36.60,36.65,36.25,36.45,-0.05,1509.0,0.000000,NaN,35.008245,30.407097,NaN,-1.565251e-01,48.837209


In [11]:
#MACD
def EMA(index,N):
    S='EMA_'+str(N)
    ans=((float(df1.at[index,'收盤價'])*2)+(N-1)*(float(df1.at[index-1,S])))/(N+1)
    return ans
def MACD(index,N,dif):
    ans=((dif*2)+(N-1)*(float(df1.at[index-1,'MACD'])))/(N+1)
    return ans
df1.loc[0,'EMA_12']=float(df1.at[0,'收盤價'])
df1.loc[0,'EMA_26']=float(df1.at[0,'收盤價'])
df1.loc[0,'MACD']=0
for i in range(1,len(df1)):
    A=EMA(i,12)
    B=EMA(i,26)
    dif=A-B
    macd=MACD(i,14,dif)
    df1.loc[i,'EMA_12']=A
    df1.loc[i,'EMA_26']=B
    df1.loc[i,'MACD']=macd
df1

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,RSI_6,RSI_12,K,D,RSI_DIF,BIAS,WMR,EMA_12,EMA_26,MACD
0,106/04/05,7385595.0,2.727916e+08,36.40,37.20,36.40,37.05,0.75,3091.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.050000,37.050000,0.000000
1,106/04/06,5074098.0,1.890247e+08,37.20,37.45,37.10,37.30,0.25,2430.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.088462,37.068519,0.002659
2,106/04/07,3786727.0,1.395635e+08,36.95,37.25,36.50,37.15,-0.15,1496.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.097929,37.074554,0.005421
3,106/04/10,978271.0,3.619003e+07,37.15,37.15,36.85,37.00,-0.15,619.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.082863,37.069032,0.006543
4,106/04/11,3063366.0,1.134365e+08,37.00,37.30,36.90,36.90,-0.10,1280.0,NaN,NaN,37.038333,45.679876,NaN,NaN,NaN,37.054730,37.056511,0.005433
5,106/04/12,4321948.0,1.596020e+08,37.25,37.50,36.70,36.85,-0.05,1886.0,64.516129,NaN,30.753457,40.704901,NaN,-4.054054e-01,59.090909,37.023233,37.041214,0.002311
6,106/04/13,2235480.0,8.236290e+07,36.90,37.05,36.75,36.75,-0.10,1101.0,23.809524,NaN,33.322560,38.244367,NaN,-4.643064e-01,68.181818,36.981197,37.019642,-0.003123
7,106/04/14,2230761.0,8.158226e+07,36.60,36.75,36.40,36.50,-0.25,1074.0,0.000000,NaN,22.216151,32.902162,NaN,-8.344654e-01,86.956522,36.907167,36.981150,-0.012571
8,106/04/17,2460829.0,8.975964e+07,36.65,36.70,36.35,36.50,0.00,1750.0,0.000000,NaN,18.514841,28.106868,NaN,-5.643121e-01,80.000000,36.844526,36.945510,-0.024359
9,106/04/18,3836886.0,1.397897e+08,36.60,36.65,36.25,36.45,-0.05,1509.0,0.000000,NaN,35.008245,30.407097,NaN,-1.565251e-01,48.837209,36.783830,36.908805,-0.037775
